In [4]:
import json
import jieba
import fool
from collections import Counter
path1 = './test_BIDAF.predicted.json'#模型一
path2 = './test_MLSTM.predicted.json'#模型二
path = './data_dev_raw.txt'#原始数据
jieba.load_userdict('./dict.txt')
fool.load_userdict('./dict.txt')

jieba.add_word('不含')
jieba.add_word('不到位')

def ori_data(path):
    ori = []
    with open(path) as fin:
        for lidx, line in enumerate(fin):
            sample = json.loads(line.strip())
            question = sample['question']
            quest_id = sample['question_id']
            paragraphs = sample['documents'][0]['paragraphs']
            question_type = sample['question_type']
            
            data = [quest_id,question,question_type,paragraphs]
            ori.append(data)
    return ori

#[问题id，问题，问题类型，文本段落]
def get_ans(path):
    ans = []
    with open(path) as fin:
        for lidx, line in enumerate(fin):
            sample = json.loads(line.strip())
            answers = sample['answers']
            ans.append(answers)
    return ans
   
def who_answer(s,flag):
    words, ners = fool.analysis(s)
    ansml = ''
    for ner in ners[0]:
        if flag == 1:
            if 'person' in ner[2]:
                return ner[3]
        else:
            if 'person' in ner[2]:
                ansml = ansml+ner[3]
            if 'org' in ner[2]:
                ansml = ansml+ner[3]
            if 'company' in ner[2]:
                ansml = ansml+ner[3]
    return ansml

def how_much_answer(s):
    ans = ''
    ans_temp = ''
    parts = fool.pos_cut(s)
    try:
        for i,part in enumerate(parts[0]):
            if part[1] == 'm':
                ans_temp = ans_temp+part[0]
                if parts[0][i+1][1] == 'q':
                    ans= ans+part[0]
                elif parts[0][i+1][1]=='m':
                    ans = ans+part[0]
            elif part[1] == 'q':
                if parts[0][i-1][1]=='m':
                    ans= ans+part[0]
    except:
        pass
    if ans == '':
        return ans_temp
    return ans

def n_answer(s):
    ans = ''
    parts = fool.pos_cut(s)
    for part in parts[0]:
        if part[1]=='n':
            ans = ans+part[0]
    return ans

oridata = ori_data(path)
answers1 = get_ans(path1)
answers2 = get_ans(path2) 

no_anchor = ['否','不','没','未曾','莫','非','不得']
time_anchor = ['时间','时候','哪一年','多久','哪一天','那一天','那一年']
org_anchor = ['哪个部门','哪家','哪个机构','哪个银行','哪家公司']
much_anchor = ['多少','那几','哪几','几','哪次','哪回','那次','哪些','那些']
who_anchor = ['哪些人','哪个人','谁','哪位','什么名字','叫什么']
loc_anchor =['在哪','何处','去哪','什么位置','什么地方']

In [5]:
def describle_refine(oridata,ans1,ans2):
    ans=[]
    for i,j,k in zip(oridata,ans1,ans2):
        if i[2] == 'DESCRIPTION':
            if len(j[0])==1 and len(k[0]) ==1:
                j[0] =i[3][0]
            if len(j[0])!=1:
                pass
            if len(j[0]) ==1 and len(k[0])!=1:
                j[0] = k[0]
        

In [6]:
def yesno_refine(oridata,ans1,ans2):
    for i,j,k in zip(oridata,ans1,ans2):
        if i[2] == 'yes_no':
            s=i[1]

            s=s.replace('是否','')
            s=s.replace('？','')
            s=s.replace('了','')
            s=s.replace('还','')
            s=s.replace(' ','')

            ans=j[0]
            ans=ans.replace('。','')
            ans=ans.replace('了','')
            ans=ans.replace('还','')

            if (s.find('不')>=0) and (ans.find('不')<0):
                j[0] = ["no","不是"]
            else:
                if s==ans:
                    j[0] = ["yes","是"]
                else:
                    if ans.find(s)>=0:
                        j[0] = ["yes","是"]
                    else:
                        if (ans.find('不')>=0) or (ans.find('不能')>=0) or (ans.find('只能')>=0):
                            j[0] = ["no","不是"]
                        else:
                            j[0] = ["yes","是"]
                            

In [7]:
def entity_refine(oridata,ans1,ans2):
    null_count = 0
    ans = []
    for i,j,k in zip(oridata,ans1,ans2):
        yesno_flag,who_flag,time_flag,org_flag,much_flag,who_flag,loc_flag = 0,0,0,0,0,0,0
        
#         _, ner = fool.analysis(i[1]) # question 
#         _, ners = fool.analysis(i[3][0]) # para
#         _, ners1 = fool.analysis(j[0]) # answer 1
                
            
        if i[2] == 'yes_no':
            tokens = jieba.lcut(i[3][0])
            for no in no_anchor:
                if no in tokens:
                    yesno_flag = 1
                    break
            if yesno_flag ==1:
                j[0] = ['no','不是']
            else:
                j[0] = ['yes','是']
#                 print('Question_type(%s)'%(i[2]))
#                 print('%s:%s'%(i[0],i[1]))
#                 print('paragraphs:',i[3][0])                
#                 print('Answers1:',j[0])
#                 print('Answers2:',k[0])  
#                 print('\n')
            
                
            
        if i[2] == 'entity':
            for time in time_anchor:
                if time in i[1]:
                    time_flag = 1
                    break
                    
            for org in org_anchor:
                if org in i[1]:
                    org_flag = 1
                    break 
                    
            for much in much_anchor:
                if much in i[1]:
                    much_flag = 1
                    break
            
            for who in who_anchor:
                if who in i[1]:
                    who_flag = 1
                    break
            
            for loc in loc_anchor:
                if loc in i[1]:
                    loc_flag = 1
                    break
                    
            if time_flag == 1:
                ans = ''
                _, ners = fool.analysis(i[3][0]) # para
                for m in ners[0]:
                    if 'time' in m:
                        ans = ans+ m[3]
                j[0] = ans

            elif org_flag == 1:
                ans = ''
                _, ners = fool.analysis(i[3][0],ignore=False) # para
                for ner in ners[0]:
                    if ner[2] == 'org':
                        ans = ans+ner[3]
                    if ner[2] == 'company':
                        ans = ans+ner[3]
                j[0] = ans
            
            elif much_flag == 1:
                j[0] = how_much_answer(i[3][0])
            
            elif who_flag == 1: 
                _, ner = fool.analysis(i[1]) # question 
                for m in ner[0]:
                    if 'job' in m[2]:
                        who_flag = 1
                j[0] = who_answer(i[3][0],who_flag)
                
            elif loc_anchor == 1:
                ans = ''
                _,ners = fool.analysis(i[3][0])
                for ner in ers[0]:
                    if ner[2] == 'location':
                        ans = ans+ner[3]
                j[0] = ans
                
            elif '什么' in i[1]:
                j[0] = n_answer(i[3][0])
                
            if j[0] == '':
                    j[0] = n_answer(i[3][0])  
            

In [8]:
yesno_refine(oridata,answers1,answers2)   
entity_refine(oridata,answers1,answers2)
describle_refine(oridata,answers1,answers2) 

In [12]:
s = "在此基础上，北京农商银行与政府合作推出“惠农一卡通”，"
def display(s):
    print(fool.pos_cut(s))
    print(fool.analysis(s,ignore = False))
display(s)
how_much_answer(s)

[[('在', 'p'), ('此', 'r'), ('基础', 'n'), ('上', 'f'), ('，', 'wd'), ('北京', 'ns'), ('农商', 'nz'), ('银行', 'n'), ('与', 'p'), ('政府', 'n'), ('合作', 'v'), ('推出', 'v'), ('“', 'wyz'), ('惠', 'v'), ('农', 'n'), ('一', 'm'), ('卡通', 'n'), ('”', 'wyy'), ('，', 'wd')]]
([[('在', 'p'), ('此', 'r'), ('基础', 'n'), ('上', 'f'), ('，', 'wd'), ('北京', 'ns'), ('农商', 'nz'), ('银行', 'n'), ('与', 'p'), ('政府', 'n'), ('合作', 'v'), ('推出', 'v'), ('“', 'wyz'), ('惠', 'v'), ('农', 'n'), ('一', 'm'), ('卡通', 'n'), ('”', 'wyy'), ('，', 'wd')]], [[(6, 13, 'company', '北京农商银行')]])


'一'

In [9]:
def answer_print(question_type,oridata,ans1,ans2):
    ans = []
    for i,j,k in zip(oridata,ans1,ans2):
        if (j[0] and k[0] != '。') and (j[0] == k[0]):
            if i[2] == question_type:
                print('Question_type(%s)'%(i[2]))
                print('%s:%s'%(i[0],i[1]))
                print('paragraphs:',i[3][0])
                print('Answers1:',j[0])
                print('Answers2:',k[0])
    
answer_print('entity',oridata,answers1,answers2)

Question_type(entity)
spdb1058:北京银监局是什么时候批复同意北京中关村银行开业的？
paragraphs: 2017年6月6日，北京银监局批复同意北京中关村银行开业
Answers1: 2017年6月6日
Answers2: 2017年6月6日
Question_type(entity)
spdb1469:中央是否赋予西藏的特殊优惠金融政策？
paragraphs: 2017年，西藏银行业金融机构严格贯彻落实中央第五次全国金融工作会议精神,认真执行中央赋予西藏的特殊优惠金融政策，
Answers1: 2017年，西藏银行业金融机构严格贯彻落实中央第五次全国金融工作会议精神,认真执行中央赋予西藏的特殊优惠金融政策，
Answers2: 2017年，西藏银行业金融机构严格贯彻落实中央第五次全国金融工作会议精神,认真执行中央赋予西藏的特殊优惠金融政策，
Question_type(entity)
spdb1557:构建开放型经济新体制是在哪个会议中提出的？
paragraphs: 党的十八届三中全会提出构建开放型经济新体制
Answers1: 党的十八届三中全会提出构建开放型经济新体制
Answers2: 党的十八届三中全会提出构建开放型经济新体制
Question_type(entity)
spdb1689:2014年建行辽宁省分行形成了哪四大产品体系？
paragraphs: 2014年建行辽宁省分行不断夯实客户基础，重点发展小微企业业务，形成了“速贷通”、“成长之路”、“小额贷”、“信用贷”四大产品体系，涵盖了30余种小微企业金融产品。
Answers1: 2014年建行辽宁省分行不断夯实客户基础，重点发展小微企业业务，形成了“速贷通”、“成长之路”、“小额贷”、“信用贷”四大产品体系，涵盖了30余种小微企业金融产品。
Answers2: 2014年建行辽宁省分行不断夯实客户基础，重点发展小微企业业务，形成了“速贷通”、“成长之路”、“小额贷”、“信用贷”四大产品体系，涵盖了30余种小微企业金融产品。
Question_type(entity)
spdb1768:北京农商银行与政府合作推出哪种卡？
paragraphs: 在此基础上，北京农商银行与政府合作推出“惠农一卡通”，
Answers1: 在此基础上，北京农商银行与政府合作推出“

In [161]:
filename = 'result1.csv'
with open(filename,'w') as f:        
    for i,j in zip(oridata,answers1):
        f.write(i[0]+'\t'+str(j[0])+str('\n'))
f.close()